<a href="https://colab.research.google.com/github/Sabelz/Master_Thesis_Alexander/blob/main/utils/PowerGPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Processes on the Power Plant dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master_Thesis_Alexander
!git config --global user.email "alexander.sabelstrom.1040@student.uu.se"
!git config --global user.name "Sabelz"

Mounted at /content/drive
/content/drive/MyDrive/Master_Thesis_Alexander


# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
!pip install gpytorch > \dev\null # Suppress prints
import gpytorch
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

%matplotlib inline
%load_ext autoreload
%autoreload 2
%run "datasets/power.ipynb" # Run the Power notebook
%run "utils/models.ipynb" # Run the models notebook
%run "utils/functions.ipynb" # Run the functions notebook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Master_Thesis_Alexander
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB
None

AT    False
V     False
AP    False
RH    False
PE    False
dtype: bool
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Master_Thesis_Alexander
----------------------------------------------------------
ALL MODELS: 
KISS-GP For 1D-4D data:
    Example:
      likelihood = gpytorch.likelihoods.GaussianLikeli

# Prepare The Data

In [3]:
powerPlant_data = df_PowerPlant # df_PowerPlant is defined in ../datasets/power.ipynb
# The last column is output(net hourly electrical energy output (EP)  of the plant)
x_power, y_power = powerPlant_data.iloc[:, :-1].to_numpy() , powerPlant_data.iloc[:, -1].to_numpy()


x_train_power, x_test_power, y_train_power, y_test_power = train_test_split(x_power, y_power, test_size=0.2, random_state=666)
# Transform into tensors
x_train_power, x_test_power, y_train_power, y_test_power = (
torch.from_numpy(x_train_power).float(), torch.from_numpy(x_test_power).float(),
torch.from_numpy(y_train_power).float(), torch.from_numpy(y_test_power).float())


if torch.cuda.is_available():
    x_train_power, y_train_power, x_test_power, y_test_power = (
    x_train_power.cuda(), y_train_power.cuda(), x_test_power.cuda(), y_test_power.cuda())

# Train data into batches
train_dataset_power = TensorDataset(x_train_power, y_train_power)
train_loader_power = DataLoader(train_dataset_power, batch_size=500, shuffle=True)

# Test data into batches
test_dataset_power = TensorDataset(x_test_power, y_test_power)
test_loader_power = DataLoader(test_dataset_power, batch_size=500, shuffle=False)

# Define the KISS-GP Model for 1D-4D data

In [7]:
KISSGP_likelihood_power = gpytorch.likelihoods.GaussianLikelihood() # Likelihood
KISSGP_mean_power = gpytorch.means.ConstantMean() # Mean
KISSGP_kernel_power = gpytorch.kernels.RBFKernel() # Kernel
KISSGP_model_power = KISSGP(x_train_power, y_train_power, KISSGP_likelihood_power, KISSGP_mean_power, KISSGP_kernel_power) # KISSGP defined in utils/Models.ipynb

# Train and Evaluate the KISS-GP Model

In [9]:
train(KISSGP_model_power, KISSGP_likelihood_power, x_train_power, y_train_power, training_iter= 40) # train is defined in utils/functions.ipynb

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.74 GiB. GPU 0 has a total capacty of 14.75 GiB of which 3.24 GiB is free. Process 3859 has 11.50 GiB memory in use. Of the allocated memory 7.66 GiB is allocated by PyTorch, and 3.68 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
  predictions_KISSGP_power = predict(KISSGP_model_power, KISSGP_likelihood_power, x_test_power) # predict is defined in utils/functions.ipynb

# Root Mean Square Error(RMSE)
RMSE_KISSGP_toy = mean_squared_error(y_test_power, predictions_KISSGP_power.mean, squared=False)
print("RMSE KISS-GP: ",RMSE_KISSGP_toy)


# Inducing Points Model

In [4]:
inducing_likelihood_toy = gpytorch.likelihoods.GaussianLikelihood()
inducing_mean_toy = gpytorch.means.ConstantMean()
inducing_kernel_toy = gpytorch.kernels.RBFKernel()
n_inducing_points = 150
# Generate a tensor of random indices
indices = torch.randperm(x_train_power.size(0))
# Select N random rows
inducing_points = x_train_power[indices][:n_inducing_points]
inducing_model_toy = InducingGP(inducing_likelihood_toy, inducing_mean_toy, inducing_kernel_toy, inducing_points) # InducingGP defined in utils/models.ipynb
inducing_model_toy = inducing_model_toy.to(device) # Move model to device, defined in utils/models.ipynb

# Train and Evaluate Inducing Points Model

In [5]:
# Use train_loader_power
train_ELBO(inducing_model_toy, inducing_likelihood_toy, x_train_power, y_train_power, training_iter= 1000, train_loader=train_loader_power) # train is defined in utils/functions.ipynb

In [6]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
  predictions_inducing_power = predict(inducing_model_toy, inducing_likelihood_toy, x_test_power) # predict is defined in utils/functions.ipynb

  # Root Mean Square Error(RMSE)
  # NumPy can only handle CPU tensors
  y_test_power_cpu = y_test_power.cpu()
  predictions_inducing_power_cpu = predictions_inducing_power.mean.cpu()
  RMSE_inducing_power = mean_squared_error(y_test_power_cpu, predictions_inducing_power_cpu, squared=False)
  print("RMSE Inducing Model: ",RMSE_inducing_power)

RMSE Inducing Model:  4.354558303746978
